In [16]:
!pip install networkx==2.4
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 14.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910476 sha256=8b6871a98d7f2bae1e907cd3a2150ca962f8cf8cd4030bb78c9ae5270265b9f8
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [59]:
import torch
import networkx as nx
from sklearn.preprocessing import StandardScaler
import numpy as np
from torch.nn import Module,ReLU,Dropout,Softmax,NLLLoss
from torch.optim import Adam

In [3]:
GraphDataset = nx.karate_club_graph()
scale = StandardScaler()


In [4]:
[GraphDataset.nodes[x]['club'] for x in GraphDataset.nodes]

['Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Officer',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Mr. Hi',
 'Officer',
 'Officer',
 'Mr. Hi',
 'Mr. Hi',
 'Officer',
 'Mr. Hi',
 'Officer',
 'Mr. Hi',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer',
 'Officer']

In [5]:
# Mr. Hi => False
# officer => True
labels = np.asarray([GraphDataset.nodes[x]['club']!='Mr. Hi' for x in GraphDataset.nodes]).astype(np.int64)

In [47]:
def CreateAdjacencyList(GraphDataset):

  AdjacencyList = nx.to_scipy_sparse_matrix(GraphDataset).tocoo() #(rowindex,colindex,connectionIsTrueFealse)
  Row = torch.tensor(AdjacencyList.row) # Adjacency list row values (first)
  Col = torch.tensor(AdjacencyList.col) # Adjacency list col values (second)

  Graph = torch.stack([Row,Col],0)
  return Graph


In [12]:
CreateAdjacencyList(GraphDataset).shape

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
          3,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,
          7,  7,  8,  8,  8,  8,  8,  9,  9, 10, 10, 10, 11, 12, 12, 13, 13, 13,
         13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 20, 21,
         21, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27,
         27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 31,
         31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
         33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33],
        [ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31,  0,  2,
          3,  7, 13, 17, 19, 21, 30,  0,  1,  3,  7,  8,  9, 13, 27, 28, 32,  0,
          1,  2,  7, 12, 13,  0,  6, 10,  0,  6, 10, 16,  0,  4,  5, 16,  0,  1,
          2,  3,  0,  2, 30, 32, 33,  2, 33,  0,  4

torch.Size([2, 156])

In [9]:
# embeddings : Initially degree of node is taken and normalized
# Initial features of nodes are taken as embeddings
embeddings = np.array([deg for node,deg in GraphDataset.degree()])
embeddings = scale.fit_transform(embeddings.reshape(-1,1))
print(embeddings)

[[ 2.98709092]
 [ 1.15480319]
 [ 1.41655858]
 [ 0.36953702]
 [-0.41572915]
 [-0.15397376]
 [-0.15397376]
 [-0.15397376]
 [ 0.10778163]
 [-0.67748454]
 [-0.41572915]
 [-0.93923993]
 [-0.67748454]
 [ 0.10778163]
 [-0.67748454]
 [-0.67748454]
 [-0.67748454]
 [-0.67748454]
 [-0.67748454]
 [-0.41572915]
 [-0.67748454]
 [-0.67748454]
 [-0.67748454]
 [ 0.10778163]
 [-0.41572915]
 [-0.41572915]
 [-0.67748454]
 [-0.15397376]
 [-0.41572915]
 [-0.15397376]
 [-0.15397376]
 [ 0.36953702]
 [ 1.94006936]
 [ 3.24884631]]


In [48]:
class DatasetLoader():
  def __init__(self,GraphDataset):


    self.Dataset = GraphDataset
    self.X = torch.tensor(embeddings,dtype=torch.float32) # Input features
    self.y = torch.tensor(labels,dtype=torch.long) # Outputs
    self.Graph = CreateAdjacencyList(GraphDataset) # Text and image have fixed graph structure , since GNN doesn't have specific structure structure is explicitly mentioned
    # In what structure the nodes has to interact (to refine features) is explicitly mentioned
    # Stuctured algorithms work based on the index of the input features
    # since not structured seperate structure has to be given so the algorithm work based on the given graph structure
  def __call__(self):
    return {'input':self.X,'label':self.y,'Graph':self.Graph}

In [49]:
from torch_geometric.nn import GCNConv

In [50]:
class GraphNeuralNetwork(Module):

  def __init__(self):
    super().__init__()

    self.Model = torch.nn.ModuleList()

    self.Model.append(GCNConv(1,16)) # embedding as one dimension (Normaised degree)
    self.Model.append(ReLU())
    self.Model.append(Dropout())

    self.Model.append(GCNConv(16,16))
    self.Model.append(ReLU())
    self.Model.append(Dropout())

    self.Model.append(GCNConv(16,2)) # 2 classes

    self.Model.append(Softmax(dim=-1))

  def forward(self,X,Graph):

    y = X

    for layer in self.Model:

      if(isinstance(layer,GCNConv)):
        y = layer(y,edge_index = Graph)
      else:
        y = layer(y)

    return y







In [65]:
device = torch.device('cpu')
data =  DatasetLoader(GraphDataset)()
model = GraphNeuralNetwork()
model = model.to(device) 
for k in data.keys():
  data[k] = data[k].to(device)
LossFn = NLLLoss()
optimizer = Adam(model.parameters(),lr =  1e-1)


# TRAIN
EPOCHS = 250

for epc in range(EPOCHS):
  

  optimizer.zero_grad()
  pred = model(data['input'],data['Graph'])
  loss = LossFn(pred,data['label'])
  loss.backward()
  optimizer.step()
  print(epc,loss.item())


0 -0.49849414825439453
1 -0.48815441131591797
2 -0.506911039352417
3 -0.509260356426239
4 -0.48928287625312805
5 -0.4866698086261749
6 -0.5197473168373108
7 -0.537302553653717
8 -0.5005596280097961
9 -0.5084007382392883
10 -0.5425479412078857
11 -0.5348790884017944
12 -0.47218576073646545
13 -0.5013256669044495
14 -0.46591490507125854
15 -0.4819313585758209
16 -0.4678363800048828
17 -0.4494376480579376
18 -0.5330414175987244
19 -0.5285269618034363
20 -0.49834373593330383
21 -0.5068018436431885
22 -0.5214053988456726
23 -0.55475914478302
24 -0.5043678283691406
25 -0.5039044618606567
26 -0.42177286744117737
27 -0.545946478843689
28 -0.5539340972900391
29 -0.548954963684082
30 -0.47418761253356934
31 -0.49740588665008545
32 -0.4500592052936554
33 -0.4762331545352936
34 -0.603999137878418
35 -0.5352843403816223
36 -0.5424091219902039
37 -0.47363635897636414
38 -0.46092355251312256
39 -0.555195689201355
40 -0.564350962638855
41 -0.5313154458999634
42 -0.41724273562431335
43 -0.5417977571487

In [68]:
print(torch.argmax(pred,-1),data['label'])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 1]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
